In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.nasnet import NASNetLarge,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortex"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (331,331,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(331, 331))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 331, 331, 3)

In [5]:
# Creating base_model
image_shape = (331, 331, 3)                                                                                                                                                                                                                                                                                            
base_model = NASNetLarge(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 4032)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 4032)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min


[CV] ........................... C=0.01, score=0.854207, total= 1.6min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.838583, total= 1.8min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.828431, total= 1.9min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.832677, total= 1.9min
[CV] ........................... C=0.01, score=0.845697, total= 1.8min
[CV] C=1.0 ...........................................................
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.855302, total= 1.9min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.830357, total= 1.9min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  6.6min


[CV] ............................ C=0.1, score=0.894531, total= 6.5min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.889219, total= 6.5min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.903579, total= 6.5min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.889881, total= 7.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.906067, total= 7.5min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.887255, total= 7.6min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.900892, total= 8.0min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.919765, total= 8.2min
[CV] ................... C=1000000000.0, score=0.894685, total= 7.8min
[CV] ................... C=1000000000.0, score=0.920588, total= 8.1min
[CV] ................... C=1000000000.0, score=0.902559, total= 7.8min
[CV] ................... C=1000000000.0, score=0.866469, total= 7.8min
[CV] ................... C=1000000000.0, score=0.907738, total= 6.7min
[CV] ................... C=1000000000.0, score=0.913776, total= 6.9min
[CV] ................... C=1000000000.0, score=0.909543, total= 6.5min
[CV] ................... C=1000000000.0, score=0.905378, total= 6.6min
[CV] .................. C=10000000000.0, score=0.897461, total= 6.1min
[CV] .................. C=10000000000.0, score=0.905088, total= 6.0min
[CV] .................. C=10000000000.0, score=0.902559, total= 5.8min
[CV] .................. C=10000000000.0, score=0.905512, total= 5.8min
[CV] .................. C=10000000000.0, score=0.909990, total= 5.6min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 50.5min remaining:  2.0min


[CV] .................. C=10000000000.0, score=0.917740, total= 5.6min
[CV] .................. C=10000000000.0, score=0.907738, total= 5.6min
[CV] .................. C=10000000000.0, score=0.905378, total= 5.5min
[CV] .................. C=10000000000.0, score=0.920588, total= 6.2min
[CV] .................. C=10000000000.0, score=0.905567, total= 5.6min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 50.6min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([104.3850704 , 410.09595721, 870.17727594, 852.44194639,
        780.67569816, 638.25499332, 561.75293097, 507.88082831,
        493.03171685, 490.81905644, 489.15543671, 446.90816114,
        345.94493377]),
 'mean_score_time': array([0.22172806, 0.26611812, 0.22717202, 0.16676648, 0.10664508,
        0.07142038, 0.13614872, 0.07243237, 0.06748424, 0.07718911,
        0.06200366, 0.05006695, 0.03044777]),
 'mean_test_score': array([0.84135754, 0.89483031, 0.91505525, 0.91278611, 0.90982636,
        0.91051697, 0.91298343, 0.91120758, 0.90726125, 0.90538674,
        0.90469613, 0.90528808, 0.90775454]),
 'mean_train_score': array([0.85834878, 0.93423889, 0.98660433, 0.99691976, 0.99626178,
        0.99565844, 0.99724907, 0.99785166, 0.99768699, 0.99357743,
        0.99322639, 0.99356672, 0.99671041]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.84136, std: 0.00885, params: {'C': 0.01},
 mean: 0.89483, std: 0.00649, params: {'C': 0.1},
 mean: 0.91506, std: 0.00666, params: {'C': 1.0},
 mean: 0.91279, std: 0.00495, params: {'C': 10.0},
 mean: 0.90983, std: 0.00631, params: {'C': 100.0},
 mean: 0.91052, std: 0.00924, params: {'C': 1000.0},
 mean: 0.91298, std: 0.00602, params: {'C': 10000.0},
 mean: 0.91121, std: 0.00788, params: {'C': 100000.0},
 mean: 0.90726, std: 0.00568, params: {'C': 1000000.0},
 mean: 0.90539, std: 0.01449, params: {'C': 10000000.0},
 mean: 0.90470, std: 0.01391, params: {'C': 100000000.0},
 mean: 0.90529, std: 0.01488, params: {'C': 1000000000.0},
 mean: 0.90775, std: 0.00652, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 1.0} - score: 0.9151
